In [1]:
from src.pdfinsight import pdf_extractor, remove_toc, pivot_df_by_heading, df2docstore
import pandas as pd
import os

In [8]:
pd.options.display.max_rows = 15
pd.options.display.max_columns = 15

In [9]:
df = pdf_extractor("tests\sample.pdf", toc_pages = 1 )
df
# df_combined = pd.DataFrame()
# for (root,dirs,files) in os.walk('.', topdown=True):
#     for file in files:
#         # only use it on pdf docs. ignore other file type such as txt
#         if file.split(".")[-1] == "pdf":
#             # print(root+"\\"+file)
#             df = pdf_extractor(root+"\\"+file, toc_pages = 2, precision_dp = 2)
#             df_combined = pd.concat([df_combined,df])
#     break

# # reset index
# df_combined.reset_index(drop=True, inplace=True)

,file,page,block,refined_block,block_ymin_diff,block_is_list,xmin,...,font_size,font_characteristics,font,font_color,text,image,cat
0,sample.pdf,1,2,1,NaN,False,169.339996,...,72.00,0,Calibri,0,THIS IS A,,toc
1,sample.pdf,1,3,2,95.0,False,112.940002,...,72.00,0,Calibri,0,SAMPLE PDF,,toc
2,sample.pdf,2,1,3,-373.0,False,271.250000,...,11.04,0,Calibri,0,Sample PDF,,header
3,sample.pdf,2,6,5,-707.0,False,72.024002,...,15.96,16,Calibri-Bold,0,TITLE,,heading1
4,sample.pdf,2,7,6,30.0,False,72.024002,...,12.00,0,Calibri,0,"Lorem ipsum dolor sit amet, consectetur adipis...",,content
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,sample.pdf,3,16,21,-15.0,False,378.190002,...,12.00,0,Calibri,0,3956,,table
77,sample.pdf,3,14,21,59.0,False,77.664001,...,12.00,0,Calibri,0,euismod sit amet tortor.,,table
78,sample.pdf,3,15,21,-15.0,False,227.929993,...,12.00,0,Calibri,0,rhoncus semper.,,table
79,sample.pdf,3,3,15,730.0,False,258.890015,...,11.04,0,Calibri,4485572,THIS IS A FOOTER,,footer


In [10]:
df[df['cat']=='page_number']

,file,page,block,refined_block,block_ymin_diff,block_is_list,xmin,...,font_size,font_characteristics,font,font_color,text,image,cat
52,sample.pdf,2,4,4,14.0,False,273.170013,...,11.04,0,Calibri,4485572,Page 2 of 3,,page_number
80,sample.pdf,3,4,15,14.0,False,273.170013,...,11.04,0,Calibri,4485572,Page 3 of 3,,page_number


In [4]:
# we check the ymin difference between each row INSTEAD OF BLOCK. 
# this is important in determining the para_thres
# text below the para_thres would be considered to be in the same block of text (ie para) 
# seems like if those between 0 to 20 is above 70%, we can set that as para_thres
pd.DataFrame(pd.cut([idx for idx in df['ymin_round'].diff()], [-1000, -0.01,10, 20, 30, 40, 50, 1000])).value_counts(normalize=True).sort_index()

(-1000.0, -0.01]    0.0250
(-0.01, 10.0]       0.2250
(10.0, 20.0]        0.5750
(20.0, 30.0]        0.1125
(30.0, 40.0]        0.0250
(40.0, 50.0]        0.0125
(50.0, 1000.0]      0.0250
Name: proportion, dtype: float64

In [5]:
# double check the ymin_thres between REFINED_BLOCKS
# as long as our defined para_thres of 20 is the majority
# then should be safe to use that as para_thres
pd.DataFrame(pd.cut([idx for idx in df['block_ymin_diff']], [-1000, -0.01,10, 20, 30, 40, 50, 1000])).value_counts(normalize=True).sort_index()

(-1000.0, -0.01]    0.1500
(-0.01, 10.0]       0.0000
(10.0, 20.0]        0.3625
(20.0, 30.0]        0.3875
(30.0, 40.0]        0.0000
(40.0, 50.0]        0.0375
(50.0, 1000.0]      0.0625
Name: proportion, dtype: float64

In [6]:
# remove rows where cat column is marked as 'toc' 
df = remove_toc(df)
# pivot the dataframe such that for each row's text, it will
# 1) merge with previous row if they are of the same category
# 2) it will iteratively search up the rows to search for the 
#    relevant headings for the row's text
pivot_df = pivot_df_by_heading(df)
pivot_df

,file,heading1,heading2,content
0,tests/sample.pdf,TITLE,None,"Lorem ipsum dolor sit amet, consectetur adipis..."
1,tests/sample.pdf,TITLE,Maecenas eu dapibus diam.,a) Suspendisse id sem sed lacus luctus digniss...
2,tests/sample.pdf,TITLE,Proin at lorem eu,Proin at lorem eu urna volutpat dignissim vel ...


In [7]:
# set the links to be the same as the filename
link_dict = dict(zip(pivot_df.file.unique(), pivot_df.file.unique()))
# convert the pivot_df into a dictionary format suitable for ingestion by a sentence transformers model
docStore = df2docstore(pivot_df, chunk_size = 100, link_dict = link_dict)
docStore

[{'content': 'TITLE\nLorem ipsum dolor sit amet, consectetur adipiscing elit. Nam accumsan sollicitudin ullamcorper. Integer id vestibulum risus. Suspendisse odio erat, venenatis dictum mi at, porttitor cursus mauris. Mauris non leo eu nisi rhoncus lacinia bibendum vitae orci. Maecenas diam lectus, ultricies vitae enim a, maximus porta magna. Fusce posuere dolor blandit, egestas leo posuere, ornare enim. Ut dignissim iaculis leo eu euismod. Etiam pulvinar ac ex non pretium. Sed tempus in est vitae tristique. Sed ac justo ut eros gravida laoreet in vel erat. Sed finibus non ante elementum pretium. Proin leo nunc, feugiat vel nulla at, porttitor pulvinar nibh.\n1. Sed rhoncus posuere mattis.\n2. Fusce dictum nisi at faucibus vulputate.\n3. Cras massa velit, suscipit a eros et, aliquet consequat mauris. Proin maximus pharetra ligula bibendum commodo. Fusce quis neque dui. In molestie purus et turpis maximus ultricies. Morbi ultricies tincidunt tellus, ac pharetra urna consequat a. Nullam 